## 工具汇总
| 工具名称 | 类型 | 网址 |
|:------:|:----:|:----:|
|`GoogleSearch` | 调用进行网络搜索 | [https://serpapi.com/dashboard](https://serpapi.com/dashboard)|

In [ ]:
!pip install google-search-results

In [ ]:
import os
import datetime
from openai import OpenAI
from typing import Dict, Any
from dotenv import load_dotenv
from serpapi import SerpApiClient

load_dotenv('./API_KEY.env')
SERACH_API_KEY = os.getenv("SERACH_API_KEY")
DOUBAO_API_KEY = os.getenv("DOUBAO_API_KEY")
DOUBAO_LLM_URL = os.getenv("DOUBAO_LLM_URL")
# https://console.volcengine.com/ark/region:ark+cn-beijing/overview?briefPage=0&briefType=introduce&type=new
# https://serpapi.com/dashboard

def custom_print(message):
    print(f"{datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')} {message}")

def DouBaoLLM(messages: list, temperature: float = 0, model_name: str=None) -> str:
    # 定义模型
    sucess = False
    model_index = 0
    model_list = [
        'doubao-seed-2-0-pro-260215',
        'doubao-seed-1-6-250615', 
        'doubao-seed-1-6-flash-250615', 
        'doubao-seed-1-8-251228',
        'doubao-seed-2-0-code-preview-260215',
        'doubao-seed-2-0-mini-260215',
        'doubao-1-5-pro-256k-250115',
        'doubao-1-5-pro-32k-250115',
        'doubao-1-5-lite-32k-250115'
        ]
    doubao_client = OpenAI(base_url= DOUBAO_LLM_URL, api_key=DOUBAO_API_KEY)

    while not sucess and model_index < len(model_list):
        model_name = model_name if model_name else model_list[model_index]
        custom_print(f"使用模型: {model_name}")
        try:
            response = doubao_client.chat.completions.create(
                model = model_name,
                messages=messages,
                temperature=temperature
            )
            custom_print(f"模型响应完成")
            sucess = True
            return response.choices[0].message.content
        except Exception as e:
            custom_print(f"模型({model_name})调用失败: {str(e)}")
            model_index += 1
            custom_print(f"正在重试（{model_index}/{len(model_list)}）...")
    return "模型调用失败，已达到最大重试次数。"

def google_search(query, api_key):
    custom_print(f"🔎 搜索: {query}")
    try:
        params = {
                "engine": "google",
                "q": query,
                "api_key": api_key,
                "gl": "cn",  # 国家代码
                "hl": "zh-cn", # 语言代码
            }
        client = SerpApiClient(params)
        results = client.get_dict()
        if "answer_box_list" in results:
            return "\n".join(results["answer_box_list"])
        if "answer_box" in results and "answer" in results["answer_box"]:
            return results["answer_box"]["answer"]
        if "knowledge_graph" in results and "description" in results["knowledge_graph"]:
            return results["knowledge_graph"]["description"]
        if "organic_results" in results and results["organic_results"]:
            snippets = [
                f"[{i+1}] {res.get('title', '')}\n{res.get('snippet', '')}"
                for i, res in enumerate(results["organic_results"][:3])
            ]
            return "\n\n".join(snippets)
        
        return f"对不起，没有找到关于 '{query}' 的信息。"
    except Exception as e:
        return f"搜索过程中发生错误: {str(e)}"

class ToolExecutor:
    def __init__(self):
        self.tools: Dict[str, Dict[str, Any]] = {}

    def registerTool(self, name: str, description: str, func: callable):
        '''向工具箱中注册一个新工具'''
        if name in self.tools:
            print(f"警告:工具 '{name}' 已存在，将被覆盖。")
        self.tools[name] = {"description": description, "func": func}
        print(f"工具 '{name}' 已注册。")

    def getTool(self, name: str) -> callable:
        '''根据名称获取一个工具的执行函数'''
        return self.tools.get(name, {}).get("func")

    def getAvailableTools(self) -> str:
        '''获取所有可用工具'''
        return "\n".join([
            f"- {name}: {info['description']}" 
            for name, info in self.tools.items()
        ])

# ReAct范式设计

In [ ]:
import re

REACT_PROMPT_TEMPLATE = """
请注意，你是一个有能力调用外部工具的智能助手。

可用工具如下:
{tools}

请严格按照以下格式进行回应:

Thought: 你的思考过程，用于分析问题、拆解任务和规划下一步行动。
Action: 你决定采取的行动，必须是以下格式之一:
- `{{tool_name}}[{{tool_input}}]`:调用一个可用工具。
- `Finish[最终答案]`:当你认为已经获得最终答案时。
- 当你收集到足够的信息，能够回答用户的最终问题时，你必须在Action:字段后使用 Finish[最终答案] 来输出最终答案。

现在，请开始解决以下问题:
Question: {question}
History: {history}
"""

SUMMARY_PROMPT_TEMPLATE = """
基于以下历史搜索结果和观察，总结并回答原始问题: {question}

历史记录:
{history}

请直接输出最终答案总结，不要添加额外解释。
"""
    
class ReActAgent:
    def __init__(self, llm_client: DouBaoLLM, tool_executor: ToolExecutor, max_steps: int = 5):
        self.llm_client = llm_client
        self.tool_executor = tool_executor
        self.max_steps = max_steps
        
        self.history = []  # 记录对话历史
    
    def run(self, question: str):
        self.history = []
        current_step = 0

        while current_step < self.max_steps:
            current_step += 1
            custom_print(f"第 {current_step} 步")

            tools_desc = self.tool_executor.getAvailableTools()
            history_str = "\n".join(self.history)
            prompt = REACT_PROMPT_TEMPLATE.format(
                tools=tools_desc,
                question=question,
                history=history_str
            )

            # 2. 调用LLM进行思考
            messages = [{"role": "user", "content": prompt}]
            response_text = self.llm_client(messages=messages)

            if not response_text:
                custom_print("错误:LLM未能返回有效响应。")
                break

            thought, action = self._parse_output(response_text)
            if thought:
                custom_print(f"🤔 Thought: {thought}")
                custom_print(f"🤔 Action: {action}")
            if not action:
                custom_print("警告:未能解析出有效的Action，流程终止。")
                break

            # 4. 执行Action
            if action.startswith("Finish"):
                final_answer = re.match(r"Finish\[(.*)\]", action).group(1)
                custom_print(f"🎉 最终答案: {final_answer}")
                return final_answer
            
            tool_name, tool_input = self._parse_action(action)
            if not tool_name or not tool_input:
                custom_print("⚠️ 警告:Action格式不正确，无法解析工具名称或输入，流程终止。")
                continue
            custom_print(f"🎬 行动: {tool_name}[{tool_input}]")
            
            tool_function = self.tool_executor.getTool(tool_name)
            if not tool_function:
                observation = f"错误:未找到名为 '{tool_name}' 的工具。"
            else:
                observation = tool_function(tool_input)
            custom_print(f"👀 观察: {observation}")
            self.history.append(f"Action: {action}")
            self.history.append(f"Observation: {observation}")

        # 达到最大步数后，总结历史结果
        custom_print("已达到最大步数，正在总结历史结果...")
        history_str = "\n".join(self.history)
        summary_prompt = SUMMARY_PROMPT_TEMPLATE.format(
            question=question,
            history=history_str
        )
        summary_messages = [{"role": "user", "content": summary_prompt}]
        final_summary = self.llm_client(messages=summary_messages)
        custom_print(f"🎉 总结答案: {final_summary}")
        return final_summary

    def _parse_output(self, text: str):
        '''解析LLM的输出，提取Thought和Action'''
        thought_match = re.search(r"Thought:\s*(.*?)(?=\nAction:|$)", text, re.DOTALL)
        action_match = re.search(r"Action:\s*(.*?)$", text, re.DOTALL)
        thought = thought_match.group(1).strip() if thought_match else None
        action = action_match.group(1).strip() if action_match else None
        return thought, action

    def _parse_action(self, action_text: str):
        '''解析Action字符串，提取工具名称和输入'''
        pattern = r"^(.+?)\s*\[([^\]]*)\]"
        match = re.match(pattern, action_text, re.DOTALL)
        if match:
            return match.group(1), match.group(2)
        return None, None

# exampleMessages = [
#     {"role": "system", "content": "You are a helpful assistant that writes Python code."},
#     {"role": "user", "content": "写一个快速排序算法"}]
# print(DouBaoLLM(exampleMessages, temperature=0.5))

# search_result = google_search("Python是什么", SERACH_API_KEY)
# print(search_result)
if __name__ == "__main__":
    toolExecutor = ToolExecutor()
    toolExecutor.registerTool(
        name="Google Search",
        description="使用Google搜索引擎查询信息",
        func=lambda query: google_search(query, SERACH_API_KEY)
    )
    react_agnet = ReActAgent(llm_client=DouBaoLLM, tool_executor=toolExecutor, max_steps=1)
    question = "华为最新Pura系列手机主要卖点是什么？"
    react_agnet.run(question)

# Plane and Solve范式

In [ ]:
import re
import ast

PLANNER_PROMPT_TEMPLATE = """
你是一个顶级的AI规划专家。你的任务是将用户提出的复杂问题分解成一个由多个简单步骤组成的行动计划。
请确保计划中的每个步骤都是一个独立的、可执行的子任务，并且严格按照逻辑顺序排列。
你的输出必须是一个Python列表，其中每个元素都是一个描述子任务的字符串，任务数量尽量控制在2-3个内。

问题: {question}

请严格按照以下格式输出你的计划,```python与```作为前后缀是必要的:
```python
["步骤1", "步骤2", "步骤3", ...]
```
"""

STEP_EXECUTOR_SYSTEM_PROMPT = """
你是一个严谨的执行专家。现在你要完成一个大计划中的**单个步骤**。

可用工具：
{tools}

使用格式（必须严格遵守）：
Thought: 你的推理过程
Action: 工具调用格式为 tool_name[参数]   或   Finish[本步骤最终答案]

规则：
1. 只能处理当前步骤，不要越界回答其他步骤
2. 需要外部信息时，必须先使用工具
3. 得到足够信息后，必须使用 Finish[答案] 结束本步骤
4. 答案要简洁、准确、直接
"""

STEP_EXECUTOR_USER_TEMPLATE = """
原始问题：{question}

完整计划：
{plan}

历史步骤结果：
{history}

当前要完成的步骤：
{current_step}

请开始执行这个步骤。
"""

SUMMARY_PROMPT_TEMPLATE = """
基于以下历史搜索结果和观察，总结并回答原始问题: {question}

历史记录:
{history}

请直接输出最终答案总结，不要添加额外解释。
"""

class Planner:
    def __init__(self, llm_client):
        self.llm_client = llm_client

    def plan(self, question: str) -> list[str]:
        '''根据用户的问题生成一个行动计划，返回一个步骤列表'''
        prompt = PLANNER_PROMPT_TEMPLATE.format(question=question)
        
        messages = [{"role": "user", "content": prompt}]
        custom_print(f"🔍 生成计划的提示已准备好，正在调用LLM...")
        response_text = self.llm_client(messages=messages) or ""
        custom_print(f"✅ 计划已生成:\n{response_text}")
        
        try:
            plan_str = response_text.split("```python")[1].split("```")[0].strip()
            plan = ast.literal_eval(plan_str)
            return plan if isinstance(plan, list) else []
        except (ValueError, SyntaxError, IndexError) as e:
            custom_print(f"❌ 解析计划时出错: {e}")
            custom_print(f"原始响应: {response_text}")
            return []
        except Exception as e:
            custom_print(f"❌ 解析计划时发生未知错误: {e}")
            return []

class Executor:
    def __init__(self, llm_client: DouBaoLLM, tool_executor: ToolExecutor, **kwargs):
        self.llm_client = llm_client
        self.tool_executor = tool_executor
        self.max_repeats = kwargs.get("max_repeats", 2)

    def _parse_thought_action(self, text: str):
        thought_match = re.search(r'Thought:\s*(.*?)(?=Action:|$)', text, re.DOTALL | re.IGNORECASE)
        action_match = re.search(r'Action:\s*(.*?)$', text, re.DOTALL | re.IGNORECASE)
        
        thought = thought_match.group(1).strip() if thought_match else None
        action = action_match.group(1).strip() if action_match else None
        
        return thought, action

    def _parse_tool_call(self, action: str):
        # 支持两种格式：ToolName[args] 或 Finish[答案]
        action = action.strip()
        if action.startswith("Finish[") and action.endswith("]"):
            return "Finish", action[7:-1].strip()
        
        match = re.match(r'^(\w+(?:\s+\w+)*)\[(.+)\]$', action.strip())
        if match:
            tool_name = match.group(1).strip()
            args = match.group(2).strip()
            return tool_name, args
        return None, None

    def execute_step(self, question: str, plan: list, history: str, current_step: str):
        '''执行单步，并对工具进行调用'''
        step_messages = [
            {"role": "system", "content": STEP_EXECUTOR_SYSTEM_PROMPT.format(
                tools=self.tool_executor.getAvailableTools()
            )},
            {"role": "user", "content": STEP_EXECUTOR_USER_TEMPLATE.format(
                question=question,
                plan="\n".join(plan),
                history=history or "（无历史）",
                current_step=current_step
            )}
        ]

        iteration = 0
        observations = []
        while iteration < self.max_repeats:
            iteration += 1
            custom_print(f"第 {iteration} 次思考...")

            response = self.llm_client(step_messages)
            if not response:
                return "[执行失败：模型无响应]"

            thought, action = self._parse_thought_action(response)
            if thought:
                custom_print(f"  Thought: {thought[:120]}{'...' if len(thought)>120 else ''}")

            if not action:
                custom_print("  !! 没有找到 Action，提前结束该步骤")
                return "[无有效 Action]"

            tool_name, tool_input = self._parse_tool_call(action)

            if tool_name == "Finish":
                custom_print(f"  ✓ 步骤完成：{tool_input}")
                return tool_input

            if not tool_name or not tool_input:
                custom_print(f"  !! Action 格式错误：{action}")
                continue

            tool_func = self.tool_executor.getTool(tool_name)
            if not tool_func:
                observation = f"工具 {tool_name} 不存在"
            else:
                try:
                    observation = str(tool_func(tool_input))
                    observations.append(observation)
                except Exception as e:
                    observation = f"工具执行异常：{str(e)}"

            custom_print(f"  → {tool_name}[{tool_input}] → {observation[:100]}{'...' if len(observation)>100 else ''}")

            # 把本次交互加入上下文
            step_messages.append({"role": "assistant", "content": response})
            step_messages.append({"role": "system", "content": f"Observation: {observation}"})

        custom_print(f"  ⚠️ 达到单步最大迭代次数，直接总结搜索结果")
        history_str = "\n".join(observations)
        summary_prompt = SUMMARY_PROMPT_TEMPLATE.format(
            question=question,
            history=history_str
        )
        summary_messages = [{"role": "user", "content": summary_prompt}]
        final_summary = self.llm_client(messages=summary_messages)
        custom_print(f"🎉 总结答案: {final_summary}")
        return final_summary

    def execute(self, question: str, plan: list[str]) -> str:
        history = ""
        results = []

        custom_print("--- 开始逐步执行计划 ---")
        for i, step in enumerate(plan, 1):
            custom_print(f"\n ---步骤 {i}/{len(plan)}: {step}")
            step_result = self.execute_step(
                question=question,
                plan=plan,
                history=history,
                current_step=step
            )
            results.append(f"---步骤 {i} {step}:\n{step_result}")
            history += f"---步骤 {i}: {step} 结果: {step_result}\n"
            custom_print(f"→ 结果: {step_result[:200]}{'...' if len(step_result)>200 else ''}")
        final_answer = "\n\n".join(results)
        return final_answer

class PlanAndSolveAgent:
    def __init__(self, llm_client, tool_executor, **kwargs):
        self.llm_client = llm_client
        self.tool_executor = tool_executor
        self.planner = Planner(self.llm_client)
        self.executor = Executor(self.llm_client, 
                                 self.tool_executor,
                                 **kwargs)

    def run(self, question: str):
        custom_print(f"问题：{question}\n")
        
        plan = self.planner.plan(question)
        if not plan:
            return "无法生成有效计划"

        custom_print("生成的计划：")
        for i, s in enumerate(plan, 1):
            custom_print(f"  {i}. {s}")

        final = self.executor.execute(question, plan)
        custom_print("最终输出：")
        print("-"*60, f"\n{final}\n" + "-"*60)
        return final

if __name__ == "__main__":
    tool_executor = ToolExecutor()
    tool_executor.registerTool(
        name="Google Search",
        description="使用Google搜索引擎查询信息",
        func=lambda query: google_search(query, SERACH_API_KEY)
    )
    # 加快速度直接 repeats=1
    PlanAndSolveAgent(llm_client=DouBaoLLM, 
                      tool_executor=tool_executor, 
                      max_repeats=1).run("华为Pura80手机和Pura70之间有哪些差异？")

工具 'Google Search' 已注册。
2026-02-27 21:30:57 问题：华为Pura80手机和Pura70之间有哪些差异？

2026-02-27 21:30:57 🔍 生成计划的提示已准备好，正在调用LLM...
2026-02-27 21:30:57 使用模型: doubao-seed-2-0-pro-260215
2026-02-27 21:31:14 模型响应完成
2026-02-27 21:31:14 ✅ 计划已生成:
```python
["采集华为Pura80和Pura70官方公开的全维度参数信息，涵盖性能配置、屏幕参数、影像系统、续航充电、机身规格、售价、独有功能等所有对比维度", "将两款手机的同维度参数进行逐项比对，标记出所有存在不同的差异项，明确每项差异的具体内容", "将所有差异项按所属类别归类整理，输出结构化的两款手机差异汇总结果"]
```
2026-02-27 21:31:14 生成的计划：
2026-02-27 21:31:14   1. 采集华为Pura80和Pura70官方公开的全维度参数信息，涵盖性能配置、屏幕参数、影像系统、续航充电、机身规格、售价、独有功能等所有对比维度
2026-02-27 21:31:14   2. 将两款手机的同维度参数进行逐项比对，标记出所有存在不同的差异项，明确每项差异的具体内容
2026-02-27 21:31:14   3. 将所有差异项按所属类别归类整理，输出结构化的两款手机差异汇总结果
2026-02-27 21:31:14 --- 开始逐步执行计划 ---
2026-02-27 21:31:14 
 ---步骤 1/3: 采集华为Pura80和Pura70官方公开的全维度参数信息，涵盖性能配置、屏幕参数、影像系统、续航充电、机身规格、售价、独有功能等所有对比维度
2026-02-27 21:31:14 第 1 次思考...
2026-02-27 21:31:14 使用模型: doubao-seed-2-0-pro-260215
2026-02-27 21:31:26 模型响应完成
2026-02-27 21:31:26   Thought: 当前需要采集华为Pura80和Pura70的官方全维度参数信息，暂无相关数据储备，需要通过搜索引擎查询两款产品对应维度的官方公开